# Introduction

### Inference Question:
How is the likelihood of contracting heart disease is impacted by 6 different health factors?

### The Heart Disease Dataset
The dataset that we will be using for this project is one related to heart disease and the factors that contribute towards it. Different factors such as blood pressure, cholesterol and even age can have great impacts on the risk of contracting heart disease and so this data set provides past details of patients with attributes they had and whether or not they were healthy or not. The link to this dataset is included below, but in this project we will be reading the data directly from the web, and so there is no need to have the file saved locally.

https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/

From the above link, the data from the Cleveland dataset will be used. The original dataset contains 76 factors, however, the processed data, which is what will be used in our project, contains 14 factors. We will further filter this data to be the 6 factors that we will be examining, which are listed below.

#### The Health Factors We Are Focusing On:
- age -> the age of the patient
- sex -> the sex of the patient
- pain.type -> the type of pain experienced by the patient (angina [1], abnormal angina [2], not angina [3], or asymptomatic [4])
- restbps -> the resting blood pressure of the patient
- cholesterol -> the cholesterol level of the patient
- maxbpm -> the maximum heart rate of the patient

We have chosen these specific factors for our analysis, as these are the factors we think would have the greatest influence on whether or not a patient potentially has heart disease. We also think that that heart disease would affect the women more than the men, and thus have included the sex variable, to see if this hypothesis holds true. 

The factors that were omitted, were a boolean value of fasting blood sugar being less than 120, resting ecg, a boolean to determine if the angina was exercise induced, oldpeak, slope, number of vessels coloured, and thalassemia. Some of these predictors were excluded due to their uncertain nature, in terms of measurement units and missing data, such as the number of vessels coloured. One debatable predictor is the resting ecg, which measures the electrical activity in the heart, however we deemed that the resting blood pressure would be a better indicator of heart activity and decided to use that instead.
______________________________________________________________________________________________________________________________________________________

# Wrangling the Data

#### Loading Tidyverse
First we need to load the tidyverse library into R so that it is possible to wrangle the data and only select the factors that we will be making use of in this project.

In [2]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

#### Reading and Selecting the Data
We can make use of the `read_csv` function to extract the data from the web, and since there are no column names in the dataset, we will have to set `col_names = FALSE` and make a vector with the desired column names.

The dataset also classifies sick patients in three different categories, however for this analysis we will consider all of them as one category, and so we set the `health` column's values 2, 3, and 4, to 1 which signifies contracted-heart-disease, while 0 signifies a healthy patient. This allows us to more easily categorise whether or not the patient contracts a heart disease, regardless of the severity.

In [3]:
heart_data<- read_csv(url("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"), col_names = FALSE) |>
    select(X1, X2, X3, X4, X5, X8, X14)
colnames(heart_data) <- make.names(c("age", "sex", "pain.type", "restbps", "cholesterol", "maxbpm", "health"))

heart_data["health"][heart_data["health"] == 2 | heart_data["health"] == 3 | heart_data["health"] == 4] <- 1
heart_data

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,pain.type,restbps,cholesterol,maxbpm,health
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
63,1,1,145,233,150,0
67,1,4,160,286,108,1
67,1,4,120,229,129,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮
57,1,4,130,131,115,1
57,0,2,130,236,174,1
38,1,3,138,175,173,0


To more clearly see why we've decided to group all the sick patients together, we have created a table below to show how the data is split, and that if there were 4 different categories of sickness it would create unnecessary confusion and problems.

In [4]:
health_freq <- table(heart_data$health)
rownames(health_freq) = c("Healthy", "Sick")
"# of Patients with and without Heart Disease"
health_freq

[1] "# of Patients with and without Heart Disease"


Healthy    Sick 
    164     139 

# Analysis of Data

#### Splitting the Dataset into Training and Testing Data
We have chosen to select 75% of the data to be in the training data, while the other 25% will be used as testing data later on. We have also selected a seed to use, so that the results are reproducible in the future.

In [5]:
set.seed(1234)

heart_data_split <- initial_split(heart_data, prop = 0.75, strata = health)
heart_data_training <- training(heart_data_split)